In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
import numpy as np 
import pandas as pd
from collections import defaultdict
import time
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
from collections import Counter
from sklearn.model_selection import KFold,train_test_split,GridSearchCV
from sklearn.linear_model import ElasticNet,LinearRegression,ElasticNetCV
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
import xgboost as xgb


pd.set_option('max_columns',135)
pd.set_option('max_rows',200)
sns.set(color_codes=True)


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
ROUTE = '/content/drive/My Drive/Colab Notebooks/Real_Estate_AVM/data/'

training_data_dummy = pd.read_csv(ROUTE+'training_data_dummy2.csv') #(37687, 142)
print(training_data_dummy.shape)
testing_data_dummy = pd.read_csv(ROUTE+'testing_data_dummy2.csv') #(6301,141)
print(testing_data_dummy.shape)

(37686, 142)
(6301, 141)


In [ ]:
def Score(true, pred):
    pe = np.abs(np.array(true) - np.array(pred))/true
    hit = [1 for i in pe if i<=0.1]
    hit_rate = round(np.sum(hit)/len(true),4)
    mape = np.mean(pe)
    if mape <1:
      s = hit_rate*10000+(1-mape)
    else:
      s = hit_rate*10000
  
      print('Watch out mape>1')
    return s, hit_rate, mape

In [ ]:
# Training Data
N_col = training_data_dummy.shape[1]
X, X_test, Y, Y_test = train_test_split(training_data_dummy.iloc[:,1:N_col], training_data_dummy.iloc[:,0], test_size=0.3, random_state=619)
print(X_test.shape)

(11306, 141)


In [ ]:
# Testing Data
X,Y = training_data_dummy.iloc[:,1:N_col],training_data_dummy.iloc[:,0]
X_test = testing_data_dummy
print(X_test.shape)

(6301, 141)


# GBR

In [ ]:
gbr = GradientBoostingRegressor(n_estimators=16000, learning_rate=0.05,max_depth=8,max_features='sqrt',
                                    min_samples_leaf=15, min_samples_split=10, 
                                   loss='huber', random_state =619)
gbr.fit(X, Y)
y_pred_gbr = gbr.predict(X_test)
# s_gbr,hr_gbr,mape_gbr = Score(Y_test,y_pred_gbr)
# print(s_gbr)


# XGBR

In [ ]:
xgbr = xgb.XGBRegressor(random_state =619,n_estimators=2300,learning_rate=0.05,max_depth=9,subsample=0.9,colsample_bytree=0.9)
# colsample_bytree=0.4603, gamma=0.0468, learning_rate=0.05, max_depth=3,min_child_weight=1.7817, n_estimators=2200,reg_alpha=0.4640, reg_lambda=0.8571,subsample=0.5213, silent=1,
xgbr.fit(X, Y)
y_pred_xgbr = xgbr.predict(X_test)
# s_xgbr,hr_xgbr,mape_xgbr = Score(Y_test,y_pred_xgbr)
# print(s_xgbr)


# LightGBM

In [ ]:
param = {'objective':'mape',
         'feature_fraction': 0.9, 
         'learning_rate': 0.05, 
         'max_bin': 250, 
         'max_depth': 80, 
         'min_data_in_leaf': 5, 
         'num_iterations': 1600, 
         'num_leaves': 480}

In [ ]:
train_data = lgb.Dataset(X, label=Y)
# test_data = lgb.Dataset(X_test, label=Y_test)
clf_lgbm = lgb.train( params =param, train_set=train_data, valid_sets = [train_data],verbose_eval=100)
y_pred_lgbm = clf_lgbm.predict(X_test, num_iteration=clf_lgbm.best_iteration)
# s_lgbm,hr_lgbm,mape_lgbm=Score(Y_test,y_pred_lgbm)
# print(s_lgbm)

# Compute Weight

In [ ]:
mape_array = np.array([mape_gbr,mape_lgbm,mape_xgbr])
x = (mape_gbr*mape_xgbr*mape_lgbm)/((mape_gbr*mape_xgbr)+(mape_xgbr*mape_lgbm)+(mape_gbr*mape_lgbm))
w_gbr, w_xgbr, w_lgbm = x/mape_gbr, x/mape_xgbr, x/mape_lgbm
print(f'weight:{[w_gbr,w_xgbr,w_lgbm]}')
print(f'{w_gbr+w_xgbr+w_lgbm}')


y_stack1 = y_pred_gbr*(1/3)+y_pred_xgbr*(1/3)+y_pred_xgbr*(1/3)
print(Score(Y_test,y_stack))
y_stack2 = y_pred_gbr*w_gbr+y_pred_xgbr*w_xgbr+y_pred_xgbr*w_lgbm
print(Score(Y_test,y_stack))

# Output


In [ ]:
result_avm = pd.read_csv(ROUTE+'result_avm.csv')
result_avm.head()

In [ ]:

y_tosave=y_stack2

result_avm['total_price'] = y_tosave

result_avm.to_csv(ROUTE+'predict_result/result_avm_stack2.csv',index=False)
result_avm.head()


# CV 5-fold get result

In [ ]:
param = {'objective':'mape',
         'feature_fraction': 0.9, 
         'learning_rate': 0.05, 
         'max_bin': 250, 
         'max_depth': 80, 
         'min_data_in_leaf': 5, 
         'num_iterations': 1600, 
         'num_leaves': 480}

In [ ]:
result_dict = defaultdict(list)
# start = time.time()
X,Y = training_data_dummy.iloc[:,1:N_col],training_data_dummy.iloc[:,0]

kf = KFold(n_splits=5)
for idx,(train_index, test_index) in enumerate(kf.split(X)):
    start = time.time()
    print(f'{idx}')
    x_train, x_test = X.iloc[train_index,:], X.iloc[test_index,:]
    y_train, y_test = Y.iloc[train_index], Y.iloc[test_index]
#1
    gbr = GradientBoostingRegressor(n_estimators=16000, learning_rate=0.05,max_depth=8,max_features='sqrt',min_samples_leaf=15, min_samples_split=10,loss='huber', random_state =619)
    gbr.fit(x_train, y_train)
    y_pred_gbr = gbr.predict(x_test)
    s_gbr,hr_gbr,mape_gbr = Score(y_test,y_pred_gbr)

#2
    xgbr = xgb.XGBRegressor(random_state =619,n_estimators=2300,learning_rate=0.05,max_depth=9,subsample=0.9,colsample_bytree=0.9)
    xgbr.fit(x_train, y_train)
    y_pred_xgbr = xgbr.predict(x_test)
    s_xgbr,hr_xgbr,mape_xgbr = Score(y_test,y_pred_xgbr)

#3 
    train_data = lgb.Dataset(x_train, label=y_train)
    test_data = lgb.Dataset(x_test, label=y_test)
    clf_lgbm = lgb.train( params =param, train_set=train_data, valid_sets = [train_data],verbose_eval=100)
    y_pred_lgbm = clf_lgbm.predict(x_test, num_iteration=clf_lgbm.best_iteration)
    s_lgbm,hr_lgbm,mape_lgbm=Score(y_test,y_pred_lgbm)

    mape_array = np.array([mape_gbr,mape_lgbm,mape_xgbr])
    x = (mape_gbr*mape_xgbr*mape_lgbm)/((mape_gbr*mape_xgbr)+(mape_xgbr*mape_lgbm)+(mape_gbr*mape_lgbm))
    w_gbr, w_xgbr, w_lgbm = x/mape_gbr, x/mape_xgbr, x/mape_lgbm
    y_stack1 = y_pred_gbr*(1/3)+y_pred_xgbr*(1/3)+y_pred_xgbr*(1/3)
    s_c1,hr_c1,mape_c1 = Score(y_test,y_stack1)
    y_stack2 = y_pred_gbr*w_gbr+y_pred_xgbr*w_xgbr+y_pred_xgbr*w_lgbm
    s_c2,hr_c2,mape_c2 = Score(y_test,y_stack2)  
    result_dict['s_gbr'].append(s_gbr)
    result_dict['s_xgbr'].append(s_xgbr)
    result_dict['s_lgbm'].append(s_lgbm)
    result_dict['s_c1'].append(s_c1)
    result_dict['s_c2'].append(s_c2)
    
    result_dict['mape_gbr'].append(mape_gbr)
    result_dict['mape_xgbr'].append(mape_xgbr)
    result_dict['mape_lgbm'].append(mape_lgbm)
    result_dict['mape_c1'].append(mape_c1)
    result_dict['mape_c2'].append(mape_c2)
    
    end = time.time()
    print(f'{end-start}s')




In [ ]:
for k,v in result_dict.items():
  u = round(np.mean(v),2)
  sd = round(np.std(v),2)
  print(f'{k}:{u},\t{sd}')

s_gbr:4581.4,	20.42
s_xgbr:4935.41,	49.75
s_lgbm:3207.53,	61.61
s_c1:5022.61,	43.46
s_c2:5023.21,	44.82
mape_gbr:0.2,	0.0
mape_xgbr:0.19,	0.01
mape_lgbm:0.27,	0.03
mape_c1:0.19,	0.01
mape_c2:0.19,	0.01


In [5]:
 result_dict2= {'mape_c1': [0.19533576343056036,0.19531914044778637,0.18239799162038733,0.17795539767525861,0.18070236043853038],
             'mape_c2': [0.19446919752847888,0.19470920582560305,0.182415749091718,0.1779829996863106,0.18076105054290356],
             'mape_gbr': [0.19655187912855998,0.20771450230052613,0.20629788203507995,0.2021310754884332,0.2047549250310021],
             'mape_lgbm': [0.24508655912995916,0.3369849238045516,0.25397092586055486,0.2551486482577094,0.24998327344402715],
             'mape_xgbr': [0.21039571107974736,0.20679496477320458,0.18857100114675548,0.1839845208936071,0.18459851071347616],
             's_c1': [4987.80466423657,4982.804680859552,5101.817602008379,5033.822044602325,5006.819297639561],
             's_c2': [4972.805530802471,4990.805290794175,5101.817584250908,5036.822017000314,5013.819238949457],
             's_gbr': [4605.8034481208715,4597.7922854977,4588.793702117965,4558.797868924512,4555.795245074969],
             's_lgbm': [3281.75491344087,3271.6630150761953,3204.7460290741396,3148.744851351743,3130.750016726556],
             's_xgbr': [4950.789604288921,4856.7932050352265,5011.811428998853,4924.816015479107,4932.815401489286]}

In [4]:
for k,v  in result_dict.items():
  print(f'{k}: {np.mean(v)}  {np.std(v)}')

mape_c1: 0.1863421307225046  0.007472250881947564
mape_c2: 0.18606764053500283  0.007100991996847497
mape_gbr: 0.20349005279672028  0.003932298401318042
mape_lgbm: 0.2682348660993604  0.034555012004073515
mape_xgbr: 0.19486894172135813  0.011374770484420843
s_c1: 5022.613657869278  43.4621104036523
s_c2: 5023.213932359466  44.8180796513423
s_gbr: 4581.396509947203  20.422349904246868
s_lgbm: 3207.5317651339005  61.61329630797834
s_xgbr: 4935.405131058278  49.745905082954565
